In [1]:
import numpy as np

In [2]:
from BaselineSolver import BaselineSolver
from IterativeSolver import IterativeSolver
from utils import run_tests, read_tests_simplex_in_ball, run_random_test

In [5]:
times = []
t_errors = []
for _ in range(10):
    run_random_test(BaselineSolver, 2, {'grid_density': np.random.randint(100, 100000)}, read_tests_simplex_in_ball, f'./tests/2d/simplex-in-ball/')

test 	37.csv	 time 	1.7590289115905762	 t_error 	2.1943424854953264e-10
test 	16.csv	 time 	6.0201756954193115	 t_error 	1.2956891115578628e-10
test 	30.csv	 time 	2.9416096210479736	 t_error 	3.5914915486046084e-10
test 	85.csv	 time 	6.414715051651001	 t_error 	4.423319488466859e-10
test 	59.csv	 time 	0.9526710510253906	 t_error 	2.679163335628232e-09
test 	4.csv	 time 	0.2741541862487793	 t_error 	7.583179306713816e-09
test 	29.csv	 time 	5.030714750289917	 t_error 	1.6890975951255882e-08
test 	51.csv	 time 	0.880232572555542	 t_error 	3.874451870444773e-09
test 	53.csv	 time 	4.466742038726807	 t_error 	3.461737563270617e-10
test 	59.csv	 time 	1.413442611694336	 t_error 	1.227384305302337e-09


In [8]:
run_tests(IterativeSolver, 5, {'initial_grid_density': 1000, 'initial_cap_radius': 0.01, 'density_of_gridpoints_in_unit_cap': 10}, 
          read_tests_simplex_in_ball, f'./tests/5d/simplex-in-ball/')

test 	42.csv	 time 	0.47463226318359375	 t_error 	0.005991183758905838
test 	27.csv	 time 	0.4421696662902832	 t_error 	0.004765292103108831
test 	13.csv	 time 	0.4449019432067871	 t_error 	0.005672798110045063
test 	97.csv	 time 	0.44222569465637207	 t_error 	0.00708444147477727
test 	100.csv	 time 	0.44463658332824707	 t_error 	0.007022137277197804
test 	58.csv	 time 	0.4424605369567871	 t_error 	0.003896208276046531
test 	22.csv	 time 	0.46076250076293945	 t_error 	0.0049537520713599514
test 	73.csv	 time 	0.4446890354156494	 t_error 	0.004901091448519801
test 	64.csv	 time 	0.4437906742095947	 t_error 	0.007424743377549836
test 	99.csv	 time 	0.44384241104125977	 t_error 	0.0025189792466657757
test 	72.csv	 time 	0.4489250183105469	 t_error 	0.004975618019899097
test 	89.csv	 time 	0.4439694881439209	 t_error 	0.0037378755855043133
test 	1.csv	 time 	0.4445760250091553	 t_error 	0.003652975711036288
test 	50.csv	 time 	0.45995092391967773	 t_error 	0.005611531283151172
test 	60.csv

KeyboardInterrupt: 

In [5]:
from utils import ball_volume, sphere_volume